# Data Preparation

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.


## Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


In [270]:
import pandas as pd
import numpy as np
import sqlite3
from IPython.display import display
pd.options.mode.chained_assignment = None

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [271]:
print("\nportfolio: " + str(portfolio.shape[0]) + " records")
display(portfolio.head(3))
print("\nprofile: " + str(profile.shape[0]) + " records")
display(profile.head(3))
print("\ntranscript: " + str(transcript.shape[0]) + " records")
display(transcript.head(3))


portfolio: 10 records


,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed



profile: 17000 records


,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN



transcript: 306534 records


,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0


## Clean Data

Before conducting the exploratory analysis, the data should be in a cleaned format. This section will flatten the data in each table, rename the `id` columns (to make it easier to know what UID it refers to), and cast the correct datatypes.

**Porfolio:**
* Flatten `channels` column
* Rename `id` to `offer_id`

In [272]:
# Get list of distinct channels
channels = list(portfolio.explode("channels").channels.unique())
print(f"distinct channels: {channels}")

# Create 1 column per channel
for channel in channels:
    portfolio[channel] = portfolio.channels.map(lambda x: 1 if channel in x else 0)
portfolio.drop(columns=["channels"], inplace=True)
    
# Preview
portfolio.head(2)

distinct channels: ['email', 'mobile', 'social', 'web']


,reward,difficulty,duration,offer_type,id,email,mobile,social,web
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1


In [273]:
# Rename column name
portfolio.rename(columns={"id": "offer_id"}, inplace=True)

# Check
print(portfolio.columns)

Index(['reward', 'difficulty', 'duration', 'offer_type', 'offer_id', 'email',
       'mobile', 'social', 'web'],
      dtype='object')


**Profile:**
* Change `None` values in `gender` to `NaN`
* Change `became_member_on` column to `date` type
    * Create  `year`, `month`, `day` columns
* Rename `id` to `customer_id`

In [274]:
# Change gender to NaN
profile["gender"] = profile.gender.map(lambda x: np.nan if x == None else x)

# Change became_member_on to date
profile["became_member_on"] = pd.to_datetime(profile.became_member_on, format='%Y%m%d')

# Create year, month, and day columns
profile["year"] = pd.DatetimeIndex(profile.became_member_on).year
profile["month"] = pd.DatetimeIndex(profile.became_member_on).month
profile["day"] = pd.DatetimeIndex(profile.became_member_on).day

# Rename id to customer_id
profile.rename(columns={"id": "customer_id"}, inplace=True)

# Preview
profile.head(2)

,gender,age,customer_id,became_member_on,income,year,month,day
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,2017,2,12
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,2017,7,15


**Transcript:**
* Change `person` to `customer_id`
* Flatten `value` column
* Create separate tables for each event: received, viewed, transaction, and completed

In [275]:
# Rename column
transcript.rename(columns={"person": "customer_id"}, inplace=True)

In [276]:
# See keys for the value column
transcript["keys"] = transcript.value.map(lambda x: str(list(x.keys())))
transcript[["event", "keys"]].value_counts()

event            keys                  
transaction      ['amount']                138953
offer received   ['offer id']               76277
offer viewed     ['offer id']               57725
offer completed  ['offer_id', 'reward']     33579
dtype: int64

Since the key depends on the event type, we can split this into 4 tables (1 for each event) before processing the `value` column.

In [277]:
# Create a table for each event
received = transcript[transcript.event == "offer received"]
viewed = transcript[transcript.event == "offer viewed"]
completed = transcript[transcript.event == "offer completed"]
transaction = transcript[transcript.event == "transaction"]

In [278]:
# Flatten value column for received
received["offer_id"] = received.value.map(lambda x: x.get("offer id"))
received.drop(columns=["value", "keys", "event"], inplace=True)
received.head(2)

,customer_id,time,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [279]:
# Flatten value column for viewed
viewed["offer_id"] = viewed.value.map(lambda x: x.get("offer id"))
viewed.drop(columns=["value", "keys", "event"], inplace=True)
viewed.head(2)

,customer_id,time,offer_id
12650,389bc3fa690240e798340f5a15918d5c,0,f19421c1d4aa40978ebb69ca19b0e20d
12651,d1ede868e29245ea91818a903fec04c6,0,5a8bc65990b245e5a138643cd4eb9837


In [280]:
# Flatten value column for transaction
transaction["amount"] = transaction.value.map(lambda x: x.get("amount"))
transaction.drop(columns=["value", "keys", "event"], inplace=True)
transaction.head(2)

,customer_id,time,amount
12654,02c083884c7d45b39cc68e1314fec56c,0,0.83
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56


In [281]:
# Flatten value column for completed
completed["offer_id"] = completed.value.map(lambda x: x.get("offer_id"))
completed["reward"] = completed.value.map(lambda x: x.get("reward"))
completed.drop(columns=["value", "keys", "event"], inplace=True)
completed.head(2)

,customer_id,time,offer_id,reward
12658,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,2906b810c7d4411798c6938adc9daaa5,2
12672,fe97aa22dd3e48c8b143116a8403dd52,0,fafdcd668e3743c1bb461111dcafc2a4,2


**Combine tables:**

See an example of a customer's transaction history to understand how to connect these events together.

In [282]:
example_customer_id = transcript.loc[0, "customer_id"]

transcript[transcript.customer_id == example_customer_id]

,customer_id,event,value,time,keys
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,['offer id']
15561,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},6,['offer id']
47582,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.89},132,['amount']
47583,78afa995795e4d85b5d9ceeca43f5fef,offer completed,{'offer_id': '9b98b8c7a33c4b65b9aebfe6a799e6d9...,132,"['offer_id', 'reward']"
49502,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 17.78},144,['amount']
53176,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},168,['offer id']
85291,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},216,['offer id']
87134,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 19.67},222,['amount']
92104,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 29.72},240,['amount']
141566,78afa995795e4d85b5d9ceeca43f5fef,transaction,{'amount': 23.93},378,['amount']


In [ ]:
# See this example for same overlapping offer fafdcd668e3743c1bb461111dcafc2a4
# transcript[(transcript.customer_id=="0fd94650b0bd46c29140b2742eed0bea")]

In [384]:
# Check out informational offers, since they have 0 reward and 0 difficulty
completed[completed.offer_id =="5a8bc65990b245e5a138643cd4eb9837"]

,customer_id,time,offer_id,reward


* The event flow is `offer received`, `offer_viewed`, `transaction`, and `offer completed`
* However, `transaction` and `offer_completed` occur at the same time
* A customer can have multiple offers at the same time
* A transaction can complete any and multiple offers (see example of consecutive rewards at the same time at index `218394` and `218395`)
* The same offer can overlap (which makes data prep difficult)
* Informational offers have no reward, difficulty, and `offer completed` events

In order to join, we will flatten all 3 datasets into 1 table with the following steps:
1. Join `portfolio` to `received` to create a base table to know the duration of each received offer
2. Join `viewed` to the base table if the time of viewing is before the offer's expiration
3. Join `transaction` to the base table if the transactions exist between the `time_received` and `time_expired` values
4. Join `completed` to the base table if the sum of transactions meets the requirement for the offer
5. Join `profile` to the base table

Use `sqlite3` to help with `LEFT JOIN` based on a [criteria range](https://stackoverflow.com/questions/30627968/merge-pandas-dataframes-where-one-value-is-between-two-others).

In [534]:
# Make the db in memory
conn = sqlite3.connect(':memory:')

# Write the tables
portfolio.to_sql('portfolio', conn, index=False)
transaction.to_sql('orders', conn, index=False)
profile.to_sql('profile', conn, index=False)
received.to_sql('received', conn, index=False)
viewed.to_sql('viewed', conn, index=False)
completed.to_sql('completed', conn, index=False)

In [535]:
# Print summary
print(f"{received.shape[0]} offers were received.")
print(f"{viewed.shape[0]} offers were viewed. {round(100 * viewed.shape[0] / received.shape[0])}% of the received.")
print(f"{completed.shape[0]} offers were completed. {round(100 * completed.shape[0] / viewed.shape[0])}% of the viewed.")

76277 offers were received.
57725 offers were viewed. 76% of the received.
33579 offers were completed. 58% of the viewed.


In [536]:
# 1. Join portfolio to received
query = """
SELECT
    r.customer_id,
    p.offer_id,
    p.offer_type,
    p.email,
    p.mobile,
    p.social,
    p.web,
    p.difficulty,
    p.reward,
    r.time AS time_received,
    p.duration,
    r.time + 24 * p.duration AS time_expired
FROM portfolio p 
INNER JOIN received r
USING (offer_id)
"""
df = pd.read_sql_query(query, conn)
df.to_sql('df', conn, index=False)

print(df.shape[0])
df.head(2)

76277


,customer_id,offer_id,offer_type,email,mobile,social,web,difficulty,reward,time_received,duration,time_expired
0,0020c2b971eb4e9188eac86d93036a77,ae264e3637204a6fb9bb56bc8210ddfd,bogo,1,1,1,0,10,10,168,7,336
1,004c5799adbf42868b9cff0396190900,ae264e3637204a6fb9bb56bc8210ddfd,bogo,1,1,1,0,10,10,336,7,504


In [537]:
# 2. Join viewed to the base table
# Assume you are always viewing the latest "version" of the offer

query = """
WITH tmp AS (
    SELECT
        df.customer_id,
        df.offer_id,
        df.time_received,
        v.time AS time_viewed,
        ROW_NUMBER() OVER(PARTITION BY df.customer_id, df.offer_id, v.time
            ORDER BY df.time_received DESC) AS rnk
    FROM df
    INNER JOIN viewed v
        ON df.customer_id = v.customer_id
        AND df.offer_id = v.offer_id
        AND df.time_received <= v.time
)
SELECT
    df.*,
    tmp.time_viewed
FROM df
LEFT JOIN tmp
    ON df.customer_id = tmp.customer_id
    AND df.offer_id = tmp.offer_id
    AND df.time_received = tmp.time_received
    AND tmp.rnk = 1
"""

df2 = pd.read_sql_query(query, conn)
df2.to_sql('df2', conn, index=False)

print(df2.shape[0])
df2.head(2)

76277


,customer_id,offer_id,offer_type,email,mobile,social,web,difficulty,reward,time_received,duration,time_expired,time_viewed
0,0020c2b971eb4e9188eac86d93036a77,ae264e3637204a6fb9bb56bc8210ddfd,bogo,1,1,1,0,10,10,168,7,336,NaN
1,004c5799adbf42868b9cff0396190900,ae264e3637204a6fb9bb56bc8210ddfd,bogo,1,1,1,0,10,10,336,7,504,NaN


In [538]:
# 3. Join transactions to the base table, but using cumulative sum
# Assume duration of offer is counted in hours
# so if you received the offer at 11am and the duration is 1, then it expires tomorrow 11am
query = """
SELECT
    df2.*,
    t.time AS time_transaction,
    sum(t.amount) OVER(PARTITION BY df2.customer_id, df2.offer_id ORDER BY t.time
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS spend_amount,
    row_number() OVER(PARTITION BY df2.customer_id, df2.offer_id, df2.time_received ORDER BY t.time) AS number_transactions,
    row_number() OVER(PARTITION BY df2.customer_id, df2.offer_id, df2.time_received ORDER BY t.time DESC) last_index,
    1 = row_number() OVER(PARTITION BY df2.customer_id, df2.offer_id, df2.time_received ORDER BY t.time DESC) or t.time is null AS is_last_transaction
FROM df2
LEFT JOIN orders t
    ON df2.customer_id = t.customer_id
    AND df2.time_received <= t.time
    AND df2.time_expired >= t.time
"""

df3 = pd.read_sql_query(query, conn)

print(df3.shape[0])
df3.head(2)

179887


,customer_id,offer_id,offer_type,email,mobile,social,web,difficulty,reward,time_received,duration,time_expired,time_viewed,time_transaction,spend_amount,number_transactions,last_index,is_last_transaction
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,discount,1,1,0,1,10,2,576,7,744,NaN,576.0,10.27,1,4,0
1,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,discount,1,1,0,1,10,2,576,7,744,NaN,660.0,22.63,2,3,0


In [539]:
# Create a column that flags which transaction qualifies them for the reward
qualify = (
    df3[df3.spend_amount >= df3.difficulty]
    .groupby(["customer_id", "offer_id", "time_received"])
    .min()
    .time_transaction
    .reset_index()
)
qualify["qualify_flag"] = 1

# Join back to transaction table
df3b = pd.merge(df3, qualify, on=["customer_id", "offer_id", "time_received", "time_transaction"], how="left")
df3b.to_sql('df3b', conn, index=False)

In [540]:
# 4. Join completed to the base table

query = """
SELECT DISTINCT
    d.customer_id,
    d.offer_id,
    d.offer_type,
    d.email,
    d.mobile,
    d.social,
    d.web,
    d.difficulty,
    d.reward,
    d.duration,
    d.spend_amount,
    d.number_transactions,
    d.is_last_transaction,
    d.time_received,
    d.time_viewed,
    d.time_transaction,
    c.time AS time_completed,
    d.time_expired
FROM df3b d
LEFT JOIN completed c 
    ON c.customer_id = d.customer_id
    AND c.offer_id = d.offer_id
    AND c.time = d.time_transaction
    AND d.qualify_flag = 1
WHERE
    c.time IS NOT NULL
    OR d.is_last_transaction = 1
"""

df4 = pd.read_sql_query(query, conn)
df4.to_sql('df4', conn, index=False)

In [541]:
# Dedupe: Remove the last transaction if the offer has an "offer completed" before the last transaction
query = """
WITH dupes AS (
    SELECT customer_id, offer_id, time_received, count(*)
    FROM df4
    GROUP BY 1, 2, 3
    HAVING count(*) > 1
)
SELECT DISTINCT
    df4.*
FROM df4
LEFT JOIN dupes
    USING (customer_id, offer_id, time_received)
WHERE NOT (dupes.customer_id IS NOT NULL AND df4.is_last_transaction = 1)
"""

df5 = pd.read_sql_query(query, conn)

print(df5.shape[0])
df5.head(2)

76277


,customer_id,offer_id,offer_type,email,mobile,social,web,difficulty,reward,duration,spend_amount,number_transactions,is_last_transaction,time_received,time_viewed,time_transaction,time_completed,time_expired
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,discount,1,1,0,1,10,2,7,10.27,1,0,576,NaN,576.0,576.0,744
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,informational,1,1,0,1,0,0,4,8.57,1,1,336,372.0,414.0,NaN,432


In [542]:
# Check for no duplicates
df5[["customer_id", "offer_id", "time_received"]].value_counts()

customer_id                       offer_id                          time_received
ffff82501cea40309d5fdd7edcca4a07  fafdcd668e3743c1bb461111dcafc2a4  0                1
555be575c91041cbaa47eb24744cc724  3f207df678b143eea3cee63160fa8bed  408              1
5565f2c4d97543038e81496ae63255ed  9b98b8c7a33c4b65b9aebfe6a799e6d9  0                1
                                                                    504              1
                                  f19421c1d4aa40978ebb69ca19b0e20d  168              1
                                                                                    ..
a92ee1678a77410cad6f44a6a7391ec9  9b98b8c7a33c4b65b9aebfe6a799e6d9  504              1
                                  f19421c1d4aa40978ebb69ca19b0e20d  576              1
a938c3fd37a74118be0f298145aed07d  0b1e1539f2cc45b7b9fa7c272da2e1d7  336              1
                                                                    408              1
0009655768c64bdeb2e877511632db8f  2906b810c7d441

In [543]:
df5.to_sql('df5', conn, index=False)

In [544]:
# 5. Join profile to the base table
# Assumption: Every record should tie to a digital account

query = """
SELECT 
    df5.*,
    p.gender,
    p.age,
    p.became_member_on,
    p.income,
    p.year,
    p.month,
    p.day
FROM df5
INNER JOIN profile p USING (customer_id)
"""

final = pd.read_sql_query(query, conn)

In [548]:
final.head()

,customer_id,offer_id,offer_type,email,mobile,social,web,difficulty,reward,duration,...,time_transaction,time_completed,time_expired,gender,age,became_member_on,income,year,month,day
0,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,discount,1,1,0,1,10,2,7,...,576.0,576.0,744,M,33,2017-04-21 00:00:00,72000.0,2017,4,21
1,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,informational,1,1,0,1,0,0,4,...,414.0,NaN,432,M,33,2017-04-21 00:00:00,72000.0,2017,4,21
2,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,informational,1,1,1,0,0,0,3,...,228.0,NaN,240,M,33,2017-04-21 00:00:00,72000.0,2017,4,21
3,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,bogo,1,1,1,1,5,5,5,...,414.0,414.0,528,M,33,2017-04-21 00:00:00,72000.0,2017,4,21
4,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,discount,1,1,1,1,10,2,10,...,528.0,528.0,744,M,33,2017-04-21 00:00:00,72000.0,2017,4,21


In [551]:
# Sanity check
print(f"{round(100*final[final.time_received>=0].shape[0] / received.shape[0])}% received.")
print(f"{round(100*final[final.time_viewed>=0].shape[0] / viewed.shape[0])}% viewed.")
print(f"{round(100*final[final.time_completed>=0].shape[0] / completed.shape[0])}% completed.")

100% received.
100% viewed.
95% completed.


We couldn't match all the `completed` offers because of the assumption that the offer `duration` is strictly in hours. It's likely that some offers were marked as complete **after** the `time_expiration` that was set, and were not counted as "complete" in my data processing steps. For example, if an offer's `duration` was `1` (representing 1 day), I interpreted it as 24 hours from the time the offer was received. Starbucks could be defining the expiration date differently. However, 95% is acceptable for data cleanliness.

In [552]:
# Save dataset for exploration
final.to_csv("data/flattened_data.csv", index=False)